风格迁移的损失函数所依赖的数据为 预训练的 CNN 网络作为特征提取器提取出的生成图像和参考图像在网络的某些层的特征序列 ，所以选择一个有效的卷积神经网络架构尤为重要。


## VGG

VGG 是一种经典的卷积神经网络（Convolutional Neural Network，CNN），由牛津大学视觉几何组（Visual Geometry Group）的研究者于2014年提出。VGG 网络以其简洁而有效的架构而闻名，被广泛用于图像识别、特征提取和迁移学习等计算机视觉任务。

![](../images/vgg1.jpg)

VGG 网络的核心特点是它的深度和简单性。它采用了连续的卷积层和池化层构成的堆叠结构，在图像分类等任务中取得了很好的性能。VGG 网络的基本架构包含了多个卷积层和池化层，其核心组成部分是重复使用 3x3 的卷积核和 2x2 的最大池化层。在实践中，VGG 网络通常有不同的深度配置，最有名的是 VGG16 和 VGG19，分别由 16 和 19 层组成。

VGG 网络的简单设计和堆叠结构使得它易于理解和实现，并且在许多视觉任务中表现出色。它也成为了迁移学习的常见选择，即通过将预训练的 VGG 网络作为特征提取器来解决其他视觉任务，如目标检测、图像风格迁移等。

在风格迁移和内容损失函数中，VGG 网络通常被用来提取图像的特征表示。通过对生成图像和参考图像在 VGG 网络的某些层的特征进行比较，可以计算内容损失和风格损失，这有助于生成具有相似内容和风格的图像。

### VGG系列模型网络的架构
在 VGG 系列模型中，网络的架构被分成了多个段（block）或部分，每个部分包含一系列的卷积层、池化层等。这些部分被称为段（blocks）或阶段（stages），它们的作用是逐渐提取更高级别的特征表示。

在 VGG 系列模型中，特别是在 VGG19 中，网络结构被划分为五个主要的段（blocks），每个段都由一系列的卷积层（conv）、激活函数层（relu）和池化层（pool）组成。

### 手动构建VGG网络模型
TensorFlow是由Google开发的一个开源机器学习框架，它提供了丰富的工具和资源，使开发者能够构建和训练机器学习模型。TensorFlow的主要特点包括：

1. **灵活性和可扩展性：** TensorFlow提供了灵活的架构，支持构建各种类型的机器学习模型，包括神经网络、深度学习模型以及传统的机器学习模型。同时，它支持在不同平台上进行部署和运行。

2. **符号式和声明式编程：** TensorFlow使用数据流图来表示计算，通过定义计算图中的节点和边来表示操作和数据流向。这种方式使得计算过程可以高度优化，并且便于并行计算。

3. **多平台支持：** TensorFlow支持在多种硬件设备上运行，包括CPU、GPU和TPU（Tensor Processing Unit）。这使得TensorFlow能够在不同的硬件上实现高效的计算。

4. **丰富的社区支持和资源：** TensorFlow拥有庞大的开发者社区，提供了大量的教程、文档、示例代码和预训练模型，使得开发者能够更轻松地使用和扩展TensorFlow框架。

5. **高级API和扩展库：** TensorFlow提供了高级API **（例如Keras）** 和多种扩展库，使得构建和训练复杂的神经网络模型变得更加简单和快速。

6. **可视化工具：** TensorFlow提供了TensorBoard等可视化工具，能够帮助开发者分析和可视化模型的训练过程和性能。

TensorFlow已经被广泛应用于各种领域，包括自然语言处理、计算机视觉、语音识别、强化学习等。它为机器学习和深度学习提供了强大的工具和基础设施，成为了业界和学术界广泛采用的重要工具之一。

In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from scipy.io import matlab

In [3]:
# 定义神经网络层的名称
# 这些层名称代表了 VGG19 模型中的卷积层（conv）、ReLU 激活函数层（relu）和池化层（pool），它们构成了整个 VGG19 模型的网络结构
# 如conv1_1，代表了VGG19模型中第一段的第一个卷积层
VGG19_LAYERS = (
    'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
    'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
    'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
    'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
    'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'conv5_4', 'relu5_4'
)

In [5]:
# 加载预训练的 VGG19 模型的权重和均值
def load_net(data_path):
    # 使用 SciPy 库中的 loadmat 函数从指定路径加载预训练的 VGG19 模型数据，加载后的数据以字典的形式存储在变量 data 中。字典的键值对代表了 MATLAB 文件中的变量名和对应的数据。
    data = matlab.loadmat(data_path) 
    '''
    # 验证所加载的数据是否包含必要的层、类别和标准化信息
    if not all(i in data for i in ('layers', 'classes', 'normalization')):
        raise ValueError("You're using the wrong VGG19 data. Please download the correct data.")
    '''
    if "normalization" in data:
        # old format, for data where
        # MD5(imagenet-vgg-verydeep-19.mat) = 8ee3263992981a1d26e73b3ca028a123
        mean_pixel = np.mean(data["normalization"][0][0][0], axis=(0, 1))
    else:
        # new format, for data where
        # MD5(imagenet-vgg-verydeep-19.mat) = 106118b7cf60435e6d8e04f6a6dc3657
        mean_pixel = data["meta"]["normalization"][0][0][0][0][2][0][0]
        
    weights = data["layers"][0]
    return weights, mean_pixel


In [ ]:
# 创建一个二维数组
arr = np.array([[1, 2, 3],
                [4, 5, 6]])

# 使用 np.transpose() 进行行列交换
transposed_arr = np.transpose(arr,(1,0))
print("原始数组：")
print(arr)
print("\n交换行列后的数组：")
print(transposed_arr)

In [ ]:
"""
    根据提供的预加载权重构建 VGG19 网络。

    参数:
    - weights: 预加载的权重数据
    - input_image: 输入图像
    - pooling: 池化方式

    返回:
    - net: 构建的 VGG19 网络
    """

def net_preloaded(weights, input_image, pooling):
    net = {}
    '''
     input_image是一个四维张量，形状为 [batch, in_height, in_width, in_channels]
     batch 是批量大小
     in_height 和 in_width 是输入图像的高度和宽度
     in_channels 是输入通道数
    '''
    current = input_image
    for i, name in enumerate(VGG19_LAYERS):
        kind = name[:4] # 卷积层（conv）、ReLU 激活函数层（relu）、池化层（pool）
        if kind == 'conv':
            # 提取权重和偏置值(新.mat)
            kernels, bias = weights[i][0][0][2][0]
            # 调整权重数据的形状
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            '''
             height 是卷积核的高度
             width 是卷积核的宽度
             in_channels 是输入通道数，即输入数据的通道数
             out_channels 是输出通道数，即卷积核的数量
            '''
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            
            # 使用权重和偏置进行卷积操作
            current = _conv_layer(current, kernels, bias)
            
        elif kind == 'relu':
            # 应用ReLU激活函数
            current = tf.nn.relu(current)
        elif kind == 'pool':
            
            # 执行池化操作
            current = _pool_layer(current, pooling)
            
        # 将每一层的输出存储到字典中
        net[name] = current
    # 如果 net 字典的长度与 VGG19_LAYERS 的层数不相等，assert断言语句会触发异常
    assert len(net) == len(VGG19_LAYERS)
    return net

In [ ]:
# 定义卷积层的计算过程
def _conv_layer(input, weights, bias):
    # 使用TensorFlow执行卷积操作,生成一个输出张量 conv，该张量包含了卷积操作的结果
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME') 
    '''
    参数解释：
    strides：设置卷积操作的步长，(1, 1, 1, 1) 表示在输入张量的四个维度上的步长均为 1。这个参数决定了在输入张量的不同维度上的卷积移动步长。
    padding：设置填充方式，'SAME' 表示使用零填充，以保持输出大小与输入大小相同。
    '''
    return tf.nn.bias_add(conv, bias) # 加偏置


In [ ]:
# 定义池化层的计算过程
def _pool_layer(input, pooling):
    if pooling == 'avg':
        # input->[batch, height, width, channels]
        # 使用平均池化方式
        return tf.nn.avg_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                padding='SAME')
    else:
        # 使用最大池化方式
        return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                padding='SAME')

均值归一化操作在深度学习中常用于预处理图像数据，其主要目的是使数据更易于处理、更好地适应模型训练，并有助于提高模型的收敛速度和稳定性。以下是进行均值归一化操作的一些原因：

1. 数据居中处理：
   - 均值归一化通过将数据减去其均值，将数据分布的中心移动到零附近，这有助于使数据更加集中在一个范围内，有助于训练模型时的数值稳定性。
2. 减少数据冗余性和相关性：
   - 对图像数据进行均值归一化可以减少数据间的冗余性和相关性，有助于提高模型的泛化能力，防止过拟合。
3. 加速模型收敛：
   - 通过归一化处理，可以使梯度下降等优化算法更快地收敛到最优解附近，加速模型的训练过程。
4. 降低数据噪声影响：
   - 均值归一化有助于降低数据中的噪声对模型训练的影响，使模型更专注于数据的真实特征。
5. 缩放数据范围：
   - 将数据缩放到相对较小的范围内（通常是均值附近），可以避免数值溢出或数值不稳定性的问题，使训练过程更加稳定。
6. 模型对数据的不变性：
   - 归一化操作有时也有助于模型对输入数据的一些变化保持不变性，使模型对一些图像的平移、缩放等变换更具有鲁棒性。

均值归一化操作是深度学习中的常见预处理步骤，能够提高模型训练的效果、稳定性和泛化能力。不过需要根据具体的数据集和任务来决定是否使用均值归一化以及如何选择合适的归一化方法。

In [ ]:
def preprocess(image):
    mean_pixel = np.array([123.68, 116.779, 103.939])
    image = image.astype(np.float32)
    # 对图像数组的三个通道分别减去均值像素
    image[:, :, 0] -= mean_pixel[0]  # 对红色通道减去均值像素
    image[:, :, 1] -= mean_pixel[1]  # 对绿色通道减去均值像素
    image[:, :, 2] -= mean_pixel[2]  # 对蓝色通道减去均值像素
    return image


def unprocess(image):
    mean_pixel = np.array([123.68, 116.779, 103.939])
    image = image.astype(np.float32)
    image[:, :, 0] += mean_pixel[0]  # 对红色通道减去均值像素
    image[:, :, 1] += mean_pixel[1]  # 对绿色通道减去均值像素
    image[:, :, 2] += mean_pixel[2]  # 对蓝色通道减去均值像素
    return image + mean_pixel


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

### 通过模块接口使用VGG19模型

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input # 这个函数是专门为 VGG 等神经网络设计的，它会将图像进行标准化，使其能够与预训练模型的训练数据相匹配，这种预处理通常会将像素值从 [0, 255] 的范围映射到 [-1, 1] 或 [0, 1] 的范围，具体取决于特定模型的要求
from PIL import Image

def preprocess(image):
    mean_pixel = np.array([123.68, 116.779, 103.939])
    # 对图像数组的三个通道分别减去均值像素
    image[:, :, 0] -= mean_pixel[0]  # 对红色通道减去均值像素
    image[:, :, 1] -= mean_pixel[1]  # 对绿色通道减去均值像素
    image[:, :, 2] -= mean_pixel[2]  # 对蓝色通道减去均值像素
    return image

def imread(path):
    img = np.array(Image.open(path)).astype(np.float32)
    if len(img.shape) == 2:
        # 若图像是灰度图（即通道数为 2），则将其转换为 3 通道的灰度图
        img = np.dstack((img, img, img))
    elif img.shape[2] == 4:
        # 若图像是带有 Alpha 通道的 PNG 图像（即通道数为 4），则丢弃 Alpha 通道，只保留 RGB 通道
        img = img[:, :, :3]
    return img

file_path = './PNG/style_1.png'  # 替换为你自己图像的文件路径
CONTENT_LAYERS = ("block4_conv2", "block5_conv2")
content_features = {}  # 存储内容图像在神经网络中各层的特征值

# 导入 VGG19 模型，不包括顶部的分类层
vgg_model = VGG19(weights='imagenet', include_top=False)
'''
for layer in vgg_model.layers:
    print(layer.name)
'''
# 获取 VGG19 模型中指定层的输出
outputs = [vgg_model.get_layer(layer).output for layer in CONTENT_LAYERS]
# 创建新模型以输出指定层的特征
feature_extraction_model = Model(inputs=vgg_model.input, outputs=outputs)

# 图像预处理
content_image = imread(file_path)
# 将图像转换为数组（NumPy array）
content_array = np.array(content_image)
print(content_array)
# 进行预处理操作
content_preprocessed = preprocess(content_array)
print(content_preprocessed)
# 在最前面添加一个维度
content_preprocessed = tf.expand_dims(content_preprocessed, axis=0)  
# 获取内容图像在神经网络中各层的特征
content_features_outputs = feature_extraction_model(content_preprocessed)

for i, layer in enumerate(CONTENT_LAYERS):
    content_features[layer] = content_features_outputs[i].numpy()
